## Tutorial

This tutorial is for offering a guidance to use the package with a one simple callable function.<br>
You only need to prepare a root path for task-based fMRI data. 


In [ ]:
from pathlib import Path
import pandas as pd

#### Example data

The example data is a mini-size version of the dataset issued by Bornstein et al., 2017 ([paper](https://www.nature.com/articles/nn.4573#abstracthttps://www.nature.com/articles/nn.4573#abstract),[openneuro](https://openneuro.org/datasets/ds001607/versions/1.0.1)).<br>
And the data includes 8 subjects and each of them has one run for Reinforcement Learning task with 180 trials. <br>
You can download the zip file from the [link]().

In [ ]:
bids_layout = "mini_bornstein2017"
report_path = "results"
Path(report_path).mkdir(exist_ok=True)

### Input data format

#### BIDS layout with original data

The original task-based fMRI data should be in **bids_layout**.<br> 
Also, you need to place preprocessed fMRI images under the *derivatives* folder, nameed as **fmriprep**.<br>
In the original layoout of sample data, there are only events files as the preprocessed images will be used.<br>

In [ ]:
!ls mini_bornstein2017

The original BIDS layout is used for locating behavioral data ("events.tsv")

In [ ]:
pd.read_table('mini_bornstein2017/sub-01/func/sub-01_task-multiarmedbandit_events.tsv').head()

#### Derivative layout from *fMRIPrep*

The package assumes that preprocessing images is done by [fMRIPrep](https://fmriprep.org/en/stable/).<br>
Please refer to "mbfmri/utils/config.py" for configuration of it.<br> You might change configuration here to apply this package to data preprocessed by other tool than fMRIPrep.

In [ ]:
!ls mini_bornstein2017/derivatives/fmriprep

In [ ]:
!ls mini_bornstein2017/derivatives/fmriprep/sub-01/func/

#### Mask images

The mask images would be located under the below folder as default.<br>
The images here are integrated into one binary image.

In [ ]:
!ls mini_bornstein2017/masks/include

### Run MB-MVPA

The next part is finally about running the MB-MVPA package.<br>
Before proceeding, you need to choose which computational model to use.<br>
Please refer to [hBayesDM](https://hbayesdm.readthedocs.io/en/v1.0.1/models.html) and check available models.<br>
Also, the data in "events.tsv" should match with input format of the corresponding model.<br>
If not and you don't want to change your original data, you can use user-defined functions to remap the data while preprocessing.<br>
Please refer to "mbfmri.preprocessing.events.py."
Generated voxel feature data and latent process signals will be saved in new derivative layout called "MB-MVPA."
As default, the MB-MVPA derivative will be under the same derivative folder as the fMRIPrep.

#### hBayesDM model

The task of the example data can be categorizeed as multi-armed bandit task, learing the probability of three cards (or bandits) by rewards or none.
The chosen computational model in the below example is a Hierarchical Bayesian Modeling of the Multi-Armed Bandit Task using 5 Parameter Model, without C (choice perseveration) but with xi (noise) and decay rate [Niv et al., 2015](https://www.jneurosci.org/content/35/21/8145).


#### Target latent process

The target latent process is the *prediction error* of chosen options, *PEchosen*, in this example. Please refer to the [available latent process list]() for other possible processes and their explanations.

#### MVPA model

For the Multi-voxel Pattern Analysis (MVPA), **ElasticNet** is used as default.<br>
The package depends on [glmnet Python package](https://github.com/civisanalytics/python-glmnet) for fitting ElasticNet.
You will get plots for lambda searching and coefficients values as the convention of employing ElasticNet.

#### Cross-validation

The *run_mbfmri* employs a cross-validation framework to secure validity.<br>
Currently, two options are available, "N-fold" for N-fold cross-valiidationa and "N-lnso" for leave-n-subjects-out.<br>
You can enter the method through *method* argument.<br>
The pearsonr correlation plot will be generated using the results from cross-validation.<br>
All the visible reports and results are integrated from the results of each fold.<br>
(You can also locate raw result of each fold in the report folder.)

#### Brain activation map

The final output and the purpose of the MB-MVPA is a brain activation pattern map attributed to the target latent process.<br>
This will be obtained by interpreting the MVPA model. For ElasticNet, it means reading coefficients of the linear layer.<br>
You can find the nii image under the "brain_map" folder in the reports.

In [ ]:
from mbfmri.core.engine import run_mbfmri


_ = run_mbfmri(analysis='mvpa',                     # name of analysis
               bids_layout='mini_bornstein2017',    # data
               mvpa_model='elasticnet',             # MVPA model
               dm_model= 'banditNarm_lapse_decay',  # computational model
               feature_name='zoom2rgrout',          # indentifier for processed fMRI data
               task_name='multiarmedbandit',       # identifier for task
               process_name='PEchosen',             # identifier for target latent process
               subjects='all',                      # list of subjects to include
               method='5-fold',                     # type of cross-validation
               report_path=report_path,             # save path for reporting results
               confounds=["trans_x", "trans_y",     # list of confounds to regress out
                          "trans_z", "rot_x",
                          "rot_y", "rot_z"],
               n_core=4,                            # number of core for multi-processing in hBayesDM
               n_thread=4,                          # number of thread for multi-threading in generating voxel features
               overwrite=True,                      # indicate if re-run and overwriting are required
               refit_compmodel=True,                # indicate if refitting comp. model is required
              )

#### Q value as target latent process

In [ ]:
_ = run_mbfmri(analysis='mvpa',                     # name of analysis
               bids_layout='mini_bornstein2017',    # data
               mvpa_model='elasticnet',             # MVPA model
               dm_model= 'banditNarm_lapse_decay',  # computational model
               feature_name='zoom2rgrout',          # indentifier for processed fMRI data
               task_name='multiarmedbandit',       # identifier for task
               process_name='Qchosen',             # identifier for target latent process
               subjects='all',                      # list of subjects to include
               method='5-fold',                     # type of cross-validation
               report_path=report_path,             # save path for reporting results
               confounds=["trans_x", "trans_y",     # list of confounds to regress out
                          "trans_z", "rot_x",
                          "rot_y", "rot_z"],
               n_core=4,                            # number of core for multi-processing in hBayesDM
               n_thread=4,                          # number of thread for multi-threading in generating voxel features
              )

### Use precalculated latent process

In [ ]:
from mbfmri.core.engine import run_mbfmri


_ = run_mbfmri(analysis='mvpa',                     # name of analysis
               bids_layout='mini_bornstein2017',    # data
               mvpa_model='elasticnet',             # MVPA model
               skip_compmodel=True,
               feature_name='zoom2rgrout',          # indentifier for processed fMRI data
               task_name='multiarmedbandit',        # identifier for task
               process_name='PrecalculatedPEchosen',# identifier for target latent process
               subjects='all',                      # list of subjects to include
               method='5-fold',                     # type of cross-validation
               report_path=report_path,             # save path for reporting results
               confounds=["trans_x", "trans_y",     # list of confounds to regress out
                          "trans_z", "rot_x",
                          "rot_y", "rot_z"],
               n_core=4,                            # number of core for multi-processing in hBayesDM
               n_thread=4,                          # number of thread for multi-threading in generating voxel features
              )

### MLP

In [ ]:
_ = run_mbfmri(analysis='mvpa',                     # name of analysis
               bids_layout='mini_bornstein2017',    # data
               mvpa_model='mlp',                    # MVPA model
               dm_model= 'banditNarm_lapse_decay',  # computational model
               feature_name='zoom2rgrout',          # indentifier for processed fMRI data
               task_name='multiarmedbandit',        # identifier for task
               process_name='Qchosen',              # identifier for target latent process
               subjects='all',                      # list of subjects to include
               method='5-fold',                     # type of cross-validation
               report_path=report_path,             # save path for reporting results
               confounds=["trans_x", "trans_y",     # list of confounds to regress out
                          "trans_z", "rot_x",
                          "rot_y", "rot_z"],
               n_core=4,                            # number of core for multi-processing in hBayesDM
               n_thread=4,                          # number of thread for multi-threading in generating voxel features
              )

### CNN

In [ ]:
_ = run_mbfmri(analysis='mvpa',                     # name of analysis
               bids_layout='mini_bornstein2017',    # data
               mvpa_model='cnn',                    # MVPA model
               dm_model= 'banditNarm_lapse_decay',  # computational model
               feature_name='zoom2rgrout',          # indentifier for processed fMRI data
               task_name='multiarmedbandit',        # identifier for task
               process_name='Qchosen',              # identifier for target latent process
               subjects='all',                      # list of subjects to include
               method='5-fold',                     # type of cross-validation
               report_path=report_path,             # save path for reporting results
               confounds=["trans_x", "trans_y",     # list of confounds to regress out
                          "trans_z", "rot_x",
                          "rot_y", "rot_z"],
               n_core=4,                            # number of core for multi-processing in hBayesDM
               n_thread=4,                          # number of thread for multi-threading in generating voxel features
              )

### Model selection

In [ ]:
from mbfmri.core.engine import run_mbfmri


_ = run_mbfmri(analysis='mvpa',                     # name of analysis
               bids_layout='mini_bornstein2017',    # data
               mvpa_model='elasticnet',             # MVPA model
               dm_model= ['banditNarm_lapse_decay', # computational model candidates
                          'banditNarm_delta',
                          'banditNarm_2par_lapse',
                          'banditNarm_4par',
                          'banditNarm_lapse',
                          'banditNarm_singleA_laps'],
               feature_name='zoom2rgrout',          # indentifier for processed fMRI data
               task_name='multiarmedbandit',        # identifier for task
               process_name='PEchosen',             # identifier for target latent process
               subjects='all',                      # list of subjects to include
               method='5-fold',                     # type of cross-validation
               report_path=report_path,             # save path for reporting results
               confounds=["trans_x", "trans_y",     # list of confounds to regress out
                          "trans_z", "rot_x",
                          "rot_y", "rot_z"],
               n_core=4,                            # number of core for multi-processing in hBayesDM
               n_thread=4,                          # number of thread for multi-threading in generating voxel features
               srefit_compmodel=True,               # indicate if reffiting comp. model is required
              )